In [3]:
import pandas as pd # Pandas
import numpy as np # Numpy
import matplotlib as mpl
import matplotlib.pyplot as plt # Matplotlibrary
import seaborn as sns # Seaborn Library
%matplotlib inline
plt.style.use("ggplot") # 设置绘图风格


# 将以下此代码输入你要画的图前可以避免无法显示中文
from pylab import *
mpl.rcParams['font.sans-serif'] = ['SimHei']

import seaborn as sns
sns.set_style({'font.sans-serif':['SimHei','Arial']})


# 预处理

## 数据处理方法

In [1]:
# 网页数据变为字符串
data = str(data,encoding='utf-8')

NameError: name 'data' is not defined

## data_Znorm(df, *cols) 标准化数据

输入：*cols：list-like，即需要正则化的列。

输出：标准化后的df

In [39]:
def data_Znorm(df, *cols):
    df_n = df.copy()
    for col in cols:
        u = df_n[col].mean()
        std = df_n[col].std()
        df_n[col + '_Zn'] = (df_n[col] - u) / std
    return(df_n)
# 创建函数，标准化数据

## map_result(df,col) 最快捷的map方法

输入：
col: 列名，函数将该列文字等分类变量转换为1，2，3，4•••无顺序含义的数值

输出：
map过后的df

In [45]:
# 最快的分类变量mapping方法
def map_result(df,col):
    df_n = df.copy()
    class_mapping = {label: idx for idx, label in enumerate(np.unique(df_n[col]))}
    df_n[col] = df_n[col].map(class_mapping)
    return df_n

## choose_object(df) 与 choose_numeric(df)

输入：待选df

输出：选择后的df

In [3]:
# 选择分类变量列
def choose_object(df):
    df_n = df.copy()
    cols = df_n.columns
    new_cols_list = [col for col in cols if str(df_n[col].dtype) == 'object']
    return df_n[new_cols_list]

# 选择数值型变量列
def choose_numeric(df):
    df_n = df.copy()
    cols = df_n.columns
    new_cols_list = [col for col in cols if str(df_n[col].dtype) != 'object']
    return df_n[new_cols_list]

## 缺失值处理

In [5]:
# 用原有数据分布fillna（）
def random_choice_fillna(df,col,seed):
    from numpy.random import default_rng
    import numpy as np
    counts = pd.Series(df[col].value_counts()/df[col].value_counts().sum())
    list1 = counts.index
    list2 = counts.values
    rng = default_rng()
    return rng.choice(list1,p=list2)


# 简单描述分析

## create_statistics_table_num(df) 创建数值型统计量

输出：统计量的dataframe

In [23]:
# 所有初步统计变量的生成 使用于numeric
"""
automatically generate a report which contains all the statistics of the factors in the DataFrame
"""
def create_statistics_table_num(df):
    df_n = df.copy()
    describe = df_n.describe() 
    # print(type(describe))
    cols = df_n.columns
    temp = pd.DataFrame(index=['range','median','variance','skew','kurt','Coef'],columns=cols)
    for col in cols:
        list_temp = []
        list_temp.append(df_n[col].max()-df_n[col].min())
        list_temp.append(df_n[col].median())
        list_temp.append(df_n[col].var())
        list_temp.append(df_n[col].skew())
        list_temp.append(df_n[col].kurt())  
        list_temp.append(df_n[col].std()/df_n[col].mean())       
        temp[col] = list_temp
    # print(temp)
    describe = pd.concat([describe,temp])
    return describe

## create_statistics_table_cat(df) 创建分类变量的统计量

输出：统计量dataframe

In [1]:
# The variation ratio is a simple measure of statistical dispersion in nominal distributions; 
# it is the simplest measure of qualitative variation.
# It is defined as the proportion of cases which are not in the mode category

In [2]:
# 统计变量的生成 适用于 catagory
"""
automatically generate a report which contains some statistics(Here:Variation Ratio) of the factors in the DataFrame

remenber: Make sure no chinese symbol or words exist when you are calculating Variation Ratio.
"""
def create_statistics_table_cat(df):
    from scipy.stats import mode
    df_n = df.copy()
    cols = df_n.columns
    print('col name\t\tmissing num\t\tmissing %')
    for string in cols:
        missing_num = df_n[string].isnull().sum()
        missing_percent = df_n[string].isnull().sum() / len(df_n[string])
        print(string+f'\t\t{missing_num}\t\t{missing_percent}')  
    temp = pd.DataFrame(index=['Variation Ratio'],columns=cols)
    for col in cols:
        list_temp = []
        list_temp.append(1-mode(df_n[col])[1][0]/len(df_n[col]))
        temp[col] = list_temp
    return temp

## auto_html_report(df, title, output_file) 创建数值型完整的html分析报告

输入：解释如下

In [1]:
# 自动生成的html文档 常适用于numeric
"""
df:DataFrame,it's better to input data with less than 15 columns for speed purpose
title: string, the title of the html like ' Yerushalayim_TRANSACTION_NUMERIC_DATA'
output_file: string, the name of the output file like 'Yerushalayim_TRANSACTION_AUTO_REPORT_NUMERIC.html'
"""
def auto_html_report(df, title, output_file):
    df_n = df.copy()
    import pandas_profiling  
    profile=df_n.profile_report(title=title)  
    profile.to_file(output_file=output_file)  

# 时间序列 

## time_series_index(df, col) 将dataframe的索引设置为时间索引

In [30]:
# 时间序列预处理 index
"""
col: the column name of the time series like "Transaction_Date"
"""
def time_series_index(df, col):
    df_n = df.copy()
    df_n.index=pd.to_datetime(df_n[col])
    return df_n

## Dickey-Fuller test 检验

In [31]:
# 移动平均图
def draw_trend(timeseries, size):
    f = plt.figure(facecolor='white')
    # 对size个数据进行移动平均
    rol_mean = timeseries.rolling(window=size).mean()
    # 对size个数据移动平均的方差
    rol_std = timeseries.rolling(window=size).std()
 
    timeseries.plot(color='blue', label='Original')
    rol_mean.plot(color='red', label='Rolling Mean')
    rol_std.plot(color='black', label='Rolling standard deviation')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()

def draw_ts(timeseries):
    f = plt.figure(facecolor='white')
    timeseries.plot(color='blue')
    plt.show()

#Dickey-Fuller test:
def teststationarity(ts):
    dftest = adfuller(ts)
    # 对上述函数求得的值进行语义描述
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return dfoutput

# 简单可视化

In [53]:
def randomcolor():
    colorArr = ['1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
    color = ""
    for i in range(6):
        color += colorArr[random.randint(0,14)]
    return "#"+color

## hist_diagram(df, col) 直方图

In [25]:
"""
直方图
method: count
remenber: If there are too many different situations, you can change figsize or aspect to make it clear enough.
"""
def hist_diagram(df, col):
    plt.figure(figsize=(15, 10))
    sns.catplot(x=col, kind="count", palette="ch:.25", aspect=2,data=df)
    plt.show() 

## kde_diagram(df, col, color) 单变量密度图

In [27]:
"""
单变量密度图
color: deeppink; dodgerblue
remember: there are various kind of colors in the website of seaborn. please check if you need more
"""
def kde_diagram(df, col, color):
    df_n = df.copy()
    sns.kdeplot(df_n[col], shade=True, color=color, label=col, alpha=.7) 

## double_hist_compare(df, sort_col, x1_col, x2_col, y_col) 分类横向直方图——双变量比较

In [29]:
"""
分类横向直方图——双变量比较
sort_col: perform ascending based on this column 
x1_col: first variable
x2_col: second variable
y_col: catagory
"""
def double_hist_compare(df, sort_col, x1_col, x2_col, y_col):
    #sns.set(style="whitegrid")  
    # Initialize the matplotlib figure  
    df_n = df.copy()
    f, ax = plt.subplots(figsize=(9, 15))  
    df_n_sorted = df_n.sort_values(sort_col, ascending=False)  

    sns.set_color_codes("pastel")  
    sns.barplot(x= x1_col, y= y_col, data=df_n_sorted,label= x1_col, color="b")  

    sns.set_color_codes("muted")  
    sns.barplot(x= x2_col, y= y_col, data=df_n_sorted,label= x2_col, color="b")  

    ax.legend(ncol=2, loc="lower right", frameon=True)  
    xlabel = "%s & %s per department" % (x1_col,x2_col)
    ax.set(ylabel="",xlabel=xlabel)  
    sns.despine(left=True, bottom=True)  

## time_series(df,col1,col2, hue) 折线图

In [33]:
"""
折线图展现时间序列
col1: the column of the time series like "Transaction_Date"
col2: what you want to show like "Approved_Amount"
hue: column to catagorize like "BU"
"""
def time_series(df,col1,col2, hue):
    df_n = df.copy()
    #sns.set(style="darkgrid") 
    plt.figure(figsize=(26,20), dpi= 80)  
    sns.set_style({'font.sans-serif':['SimHei','Arial']})
    # Plot the responses for different events and regions  
    sns.lineplot(x=col1, y=col2, hue=hue, data=data_1)  
    plt.xticks([])

## corr_matrix(df) 相关系数矩阵 

In [36]:
"""
相关系数矩阵

"""
def corr_matrix(df):
    df_n = df.copy()
    plt.figure(figsize=(12,10), dpi= 80)  
    sns.set_style({'font.sans-serif':['SimHei','Arial']})
    sns.heatmap(df_n.corr(), xticklabels=df_n.corr().columns, yticklabels=df_n.corr().columns, cmap='coolwarm', center=0, annot=True)  
    plt.title('Correlogram', fontsize=22)  

## scatter_double(df,x_col,y_col,hue_col,size_col) 二分类变量散点图

In [8]:
"""
两分类变量，两数值变量的散点图
x_col: x轴的列
y_col: y轴的列
hue_col : 拿什么列来分类
size_col : 拿什么列的数值来作为大小
"""
def scatter_double(df,x_col,y_col,hue_col,size_col):
    df_n = df.copy()
    f, ax = plt.subplots(figsize=(13, 13)) 
    sns.set_style({'font.sans-serif':['SimHei','Arial']})
    sns.despine(f, left=True, bottom=True)  
    clarity_ranking = list(set(list(df_n[hue_col])))
    sns.scatterplot(x=x_col, y=y_col,  
                    hue=hue_col, size=size_col,  
                    palette="Paired",  
                    hue_order=clarity_ranking,  
                    sizes=(1, 16), linewidth=0,  
                    data=df_n, ax=ax)  
    ax.legend(loc="upper right", frameon=True)  

## boxplot_scatter(df, x_col, y_col, hue_col) 箱线图与散点图

In [9]:
"""
箱线图+散点图
x_col: x轴的列
y_col: y轴的列
hue_col : 拿什么列来分类
"""
def boxplot_scatter(df, x_col, y_col, hue_col):
    df_n = df.copy()
    # Draw Plot  
    plt.figure(figsize=(14,10), dpi= 80)  
    sns.boxplot(x=x_col, y=y_col, data=df_n, hue=hue_col, palette="Set2")  
    sns.stripplot(x=x_col, y=y_col, data=df_n, size=4, jitter=0.05)  #color='black'

    for i in range(len(df_n[hue_col].unique())-1):  
        plt.vlines(i+.5, 10, 45, linestyles='solid', alpha=0.2)  #colors='gray'
    title = 'Box Plot of %s by %s'% (y_col, x_col)
    plt.title(title, fontsize=22)  
    plt.legend(title=hue_col)  
    plt.show()  

# 正态化检验与正态转换
#### 非所有均能转换为正态，boxcox可以接近正态

## check_normality(testData) 正太检验

In [52]:
# 判断是否为正态
import scipy
from scipy.stats import f
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
# additional packages
#from statsmodels.stats.diagnostic import lillifors
 
def check_normality(testData):
    #20<样本数<50用normal test算法检验正态分布性
    if 20<len(testData) <50:
        p_value=stats.normaltest(testData)[1]
        if p_value<0.05:
            print("use normaltest")
            print("data are not normal distributed")
            return  False
        else:
            print("use normaltest")
            print("data are normal distributed")
            return True
     
    #样本数小于50用Shapiro-Wilk算法检验正态分布性
    if len(testData) <50:
        p_value= stats.shapiro(testData)[1]
        if p_value<0.05:
            print ("use shapiro:")
            print ("data are not normal distributed")
            return  False
        else:
            print ("use shapiro:")
            print ("data are normal distributed")
            return True
     
    if len(testData) >50: 
        p_value= stats.kstest(testData,'norm')[1]
        if p_value<0.05:
            print ("use kstest:")
            print ("data are not normal distributed")
            return  False
        else:
            print ("use kstest:")
            print ("data are normal distributed")
            return True
 
 
#对所有样本组进行正态性检验
def NormalTest(list_groups):
    for group in list_groups:
        #正态性检验
        status=check_normality(group1)
        if status==False :
            return False

In [ ]:
# 计算参数
import numpy as np
from scipy import stats
  
x = np.random.randn(10000)
mu = np.mean(x, axis=0)
sigma = np.std(x, axis=0)
skew = stats.skew(x)
kurtosis = stats.kurtosis(x)

## 转换正态的方法

### data_Transform_boxcox(df, *cols) boxcox将数据尽可能往正态靠拢

In [10]:
#转换为正态
from scipy import stats
def data_Transform_boxcox(df, *cols):
    df_n = df.copy()
    for col in cols:
        min_num = df_n[col].min()
        print(min_num)
        if df_n[col].min()<=0:
            df_n[col] = df_n[col]-min_num+1
        xt, _ = stats.boxcox(df_n[col])
        df_n[col+'_Boxcox'] = xt
    return df_n

### transform_norm(df, *cols, method) 其他数据变化的方式

In [12]:
"""
method : please choose the method you want
"""
# 其他处理变换
def data_Transform(df, *cols, method):
    df_n = df.copy()
    for col in cols:
        if method == 'log':
            xt = np.log(df_n[col])
        elif method == 'sqrt':
            xt = np.sqrt(df_n[col])
        elif method == 'reciprocal':
            xt = 1/df_n[col]
        elif method == 'arcsin_sqrt':
            xt = np.arcsin(np.sqrt(df_n[col]))
        df_n[col+'_log'] = xt
    return df_n



#如果不是正态转换为正态
def transform_norm(df, *cols, method):
    df_n = df.copy()
    for col in cols:
        result=check_normality(df_n[col])
        if result == False:
            df_n[col] = data_Transform(df_n, col, method)
            return df_n

# 方差分析

In [46]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
# 单因素方差分析
def single_anova(df, y_col, x_col):
    formula = "%s~%s" % (y_col,x_col)
    model = ols(formula,df).fit()
    anovat = anova_lm(model)
    return anovat

# 双因素方差分析/多因素方差分析
# 多变量——Expense_Type+BU
def multiple_anova(df,y_col, x_col1,x_col2):
    data = df
    
    formula = '%s~%s + %s'%(y_col,x_col1,x_col2)
    anova_results = anova_lm(ols(formula,data).fit())
    return anova_results


# 多重比较
def multiple_compare(df,col1,col2):
    from statsmodels.stats.multicomp import pairwise_tukeyhsd
    result = pairwise_tukeyhsd(df[col1], df[col2])
    return result

# 特征选择

In [47]:
from sklearn.feature_selection import SelectKBest,f_classif
#selector=SelectKBest(score_func=f_classif,k=1)
#selector.fit(company_used_features[["Expense_type","date","Region","approvers","Payment_methods","BU"]],company_used_features['Approved_Amount'])

# 特征工程——个人行为数据

##  金额部分 person_amount_feature(df,id_person,amount)

In [13]:
# 个人行为数据——金额部分所有特征
"""
id_person: 员工所在列的列名
amount: 金额列的列名

输出：员工金额部分dataframe
"""

def person_amount_feature(df,id_person,amount):
    df_copy = df.copy()
    def kurt(df):
        result = df.kurt()
        return result
    def skew(df):
        df_n=df.copy()
        df_n = df[df.values>0]
        return df_n.skew()
    Employ_Behavior_Amount=df_copy.groupby(id_person)[amount].agg(["count",'sum','mean',"median","std","max","min",skew,kurt])
    Employ_Behavior_Amount=Employ_Behavior_Amount.rename(columns={'count': 'Total_Reimbursement_Times','sum':'Total_Amount','mean':'Average_Amount','std':'Std_Amount','max':'Max_Amount','min':'Min_Amount',"median":"Median_Amount","skew":"Skew_Amount","kurt":"Kurt_Amount"})
    return Employ_Behavior_Amount

##  支付方式部分 person_cash_ratio(df,id_person,payment_methods)

In [15]:
# 行为人数据，采用现金和公司卡的比例
"""
id_person: 员工所在列的列名
payment_methods: 支付方式列的列名

输出：员工支付方式部分dataframe
"""
def person_cash_ratio(df,id_person,payment_methods):
    df_copy=df.copy()
    def ratio_count_cash(df):
        result = df.value_counts()/df.count()
        try:
            number = result.loc['现金']
        except:
            number = 0
        return number

    def ratio_count_cards(df):
        result = df.value_counts()/df.count()
        try:
            number = result.loc['公司卡']
        except:
            number = 0
        return number

    # 类型部分
    Employ_Behavior_Payment=df_copy.groupby(id_person)[payment_methods].agg([ratio_count_cash,ratio_count_cards])

    Employ_Behavior_Payment=Employ_Behavior_Payment.rename(columns={'ratio_count_cash': 'Payment_Cash_Ratio','ratio_count_cards':'Payment_Card_Ratio'})
    return Employ_Behavior_Payment

## 报销数据类型特征 person_expense_feature(df,id_person,expense_type)

In [16]:
# 行为人数据——每个人的报销数据类型特征比例
"""
id_person: 员工所在列的列名
expense_type: 报销类型列的列名

输出：员工报销类型部分dataframe
"""
def person_expense_feature(df,id_person,expense_type):
    df_copy = df.copy()
    def re1(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            name = result.loc['客户娱乐']
        except:
            name = 0
        return name

    def re2(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            number = result.loc["加班餐补"]
        except:
            number = 0
        return number

    def re3(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            name = result.loc['会议']
        except:
            name = 0
        return name

    def re4(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            number = result.loc["租车费"]
        except:
            number = 0
        return number

    def re5(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            name = result.loc['住宿']
        except:
            name = 0
        return name

    def re6(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            number = result.loc["HCP支付"]
        except:
            number = 0
        return number

    def re7(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            name = result.loc['机票费']
        except:
            name = 0
        return name

    def re8(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            number = result.loc["翻译费"]
        except:
            number = 0
        return number

    def re9(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            name = result.loc['地面交通']
        except:
            name = 0
        return name

    def re10(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            number = result.loc["停车费"]
        except:
            number = 0
        return number

    def re11(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            name = result.loc['员工餐费']
        except:
            name = 0
        return name

    def re12(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            number = result.loc["运输费"]
        except:
            number = 0
        return number

    def re13(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            number = result.loc["金融其他"]
        except:
            number = 0
        return number

    def re14(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            name = result.loc['员工娱乐']
        except:
            name = 0
        return name

    def re15(df):
        result = df.value_counts(sort=True)/df.count()
        try:
            number = result.loc["其他"]
        except:
            number = 0
        return number


    # 类型部分
    Employ_Behavior_Expense_type=df_copy.groupby(id_person)[expense_type].agg([re1,re2,re3,re4,re5,re6,re7,re8,re9,re10,re11,re12,re13,re14,re15])

    Employ_Behavior_Expense_type=Employ_Behavior_Expense_type.rename(columns={'re1':'客户娱乐','re2':'加班餐补','re3':'会议','re4':'租车费','re5':'住宿','re6': 'HCP支付','re7': '机票费','re8':'翻译费','re9':'地面交通','re10':'停车费','re11':'员工餐费','re12': '运输费','re13':'金融其他','re14':'员工娱乐','re15':'其他'})
    return Employ_Behavior_Expense_type

### 每一月的报销特征向量及其距离计算（三种方法）create_month_vector(df,data1,list_cols)

In [7]:
# 生成每月的报销费用占比形成向量，考察向量之间的关系与异常值——三种方法
# 输入参数必须为索引是日期的df,data1为待插入的用户行为数据df
"""
df: 原始数据
data1:用户特征行为表
"""
def create_month_vector(df,data1,list_cols):
    company_copy = df
    groups=company_copy[['Employee_Global_ID','Approved_Amount',"Expense type"]].groupby('Employee_Global_ID')

    list_name=[]
    list_cos=[]
    n=0

    for name,group in groups:
        print(name,n)
        n=n+1
        group = pd.DataFrame(group)
        list_cos.append(rexx(group))
        list_name.append(name)

    result = pd.DataFrame(columns=['Employee_Global_ID','distance_avg'])
    result["Employee_Global_ID"]=list_name
    result["distance_avg"]=list_cos
    result = result.set_index("Employee_Global_ID",drop=True)
    result_1=pd.concat([data1,result],axis=1)
    result_1.to_csv(r"concat_final_version.csv",encoding='gbk')
    
    def rexx(df):
        total_frame=pd.DataFrame(df['Approved_Amount'].resample('M').sum().to_period('M'))#.iloc[:,0]
        """
        #动态代码改写如下(未来提升算法速度以及提升代码重用性时可以进行改写，基本语法如下)：
        #length = len(list_cols)
        #for i in range():
        #    local()["df"+str(i)]=df[df["Expense type"]==list_cols[i]]
        """
        df_1 = df[df['Expense type']=='客户娱乐']
        total_1=pd.DataFrame(df_1['Approved_Amount'].resample('M').sum().to_period('M'))
        total_1=total_1.rename(columns={'Approved_Amount':"客户娱乐"})
        df_2 = df[df['Expense type']=='加班餐补']
        total_2=pd.DataFrame(df_2['Approved_Amount'].resample('M').sum().to_period('M'))
        total_2=total_2.rename(columns={'Approved_Amount':"加班餐补"})
        df_3 = df[df['Expense type']=='会议']
        total_3=pd.DataFrame(df_3['Approved_Amount'].resample('M').sum().to_period('M'))
        total_3=total_3.rename(columns={'Approved_Amount':"会议"})
        df_4 = df[df['Expense type']=='租车费']
        total_4=pd.DataFrame(df_4['Approved_Amount'].resample('M').sum().to_period('M'))
        total_4=total_4.rename(columns={'Approved_Amount':"租车费"})
        df_5 = df[df['Expense type']=='住宿']
        total_5=pd.DataFrame(df_5['Approved_Amount'].resample('M').sum().to_period('M'))
        total_5=total_5.rename(columns={'Approved_Amount':"住宿"})

        df_6 = df[df['Expense type']=='HCP支付']
        total_6=pd.DataFrame(df_6['Approved_Amount'].resample('M').sum().to_period('M'))
        total_6=total_6.rename(columns={'Approved_Amount':"HCP支付"})
        df_7 = df[df['Expense type']=='机票费']
        total_7=pd.DataFrame(df_7['Approved_Amount'].resample('M').sum().to_period('M'))
        total_7=total_7.rename(columns={'Approved_Amount':"机票费"})
        df_8 = df[df['Expense type']=='翻译费']
        total_8=pd.DataFrame(df_8['Approved_Amount'].resample('M').sum().to_period('M'))
        total_8=total_8.rename(columns={'Approved_Amount':"翻译费"})
        df_9 = df[df['Expense type']=='地面交通']
        total_9=pd.DataFrame(df_9['Approved_Amount'].resample('M').sum().to_period('M'))
        total_9=total_9.rename(columns={'Approved_Amount':"地面交通"})
        df_10 = df[df['Expense type']=='停车费']
        total_10=pd.DataFrame(df_10['Approved_Amount'].resample('M').sum().to_period('M'))
        total_10=total_10.rename(columns={'Approved_Amount':"停车费"})

        df_11 = df[df['Expense type']=='员工餐费']
        total_11=pd.DataFrame(df_11['Approved_Amount'].resample('M').sum().to_period('M'))
        total_11=total_11.rename(columns={'Approved_Amount':"员工餐费"})
        df_12 = df[df['Expense type']=='运输费']
        total_12=pd.DataFrame(df_12['Approved_Amount'].resample('M').sum().to_period('M'))
        total_12=total_12.rename(columns={'Approved_Amount':"运输费"})
        df_13 = df[df['Expense type']=='金融其他']
        total_13=pd.DataFrame(df_13['Approved_Amount'].resample('M').sum().to_period('M'))
        total_13=total_13.rename(columns={'Approved_Amount':"金融其他"})
        df_14 = df[df['Expense type']=='员工娱乐']
        total_14=pd.DataFrame(df_14['Approved_Amount'].resample('M').sum().to_period('M'))
        total_14=total_14.rename(columns={'Approved_Amount':"员工娱乐"})
        df_15 = df[df['Expense type']=='其他']
        total_15=pd.DataFrame(df_15['Approved_Amount'].resample('M').sum().to_period('M'))
        total_15=total_15.rename(columns={'Approved_Amount':"其他"})

        total_frame_all = pd.concat([total_frame,total_1,total_2,total_3,total_4,total_5,total_6,total_7,total_8,total_9,total_10,total_11,total_12,total_13,total_14,total_15],axis=1)

        total_frame_all['客户娱乐']=total_frame_all['客户娱乐']/total_frame_all['Approved_Amount']
        total_frame_all['加班餐补']=total_frame_all['加班餐补']/total_frame_all['Approved_Amount']
        total_frame_all['会议']=total_frame_all['会议']/total_frame_all['Approved_Amount']
        total_frame_all['租车费']=total_frame_all['租车费']/total_frame_all['Approved_Amount']
        total_frame_all['住宿']=total_frame_all['住宿']/total_frame_all['Approved_Amount']

        total_frame_all['HCP支付']=total_frame_all['HCP支付']/total_frame_all['Approved_Amount']
        total_frame_all['机票费']=total_frame_all['机票费']/total_frame_all['Approved_Amount']
        total_frame_all['翻译费']=total_frame_all['翻译费']/total_frame_all['Approved_Amount']
        total_frame_all['地面交通']=total_frame_all['地面交通']/total_frame_all['Approved_Amount']
        total_frame_all['停车费']=total_frame_all['停车费']/total_frame_all['Approved_Amount']

        total_frame_all['员工餐费']=total_frame_all['员工餐费']/total_frame_all['Approved_Amount']
        total_frame_all['运输费']=total_frame_all['运输费']/total_frame_all['Approved_Amount']
        total_frame_all['金融其他']=total_frame_all['金融其他']/total_frame_all['Approved_Amount']
        total_frame_all['员工娱乐']=total_frame_all['员工娱乐']/total_frame_all['Approved_Amount']
        total_frame_all['其他']=total_frame_all['其他']/total_frame_all['Approved_Amount']


        total_frame_all=total_frame_all.fillna(0)
        result_avg=pd.DataFrame(total_frame_all.mean())
        result_avg = result_avg.T
        total_frame_all = total_frame_all.append(result_avg)
        length = len(total_frame_all)
        result_array=np.array(total_frame_all)
        #print(result_array)

        """
        算法1：距离平均
        """
        #norm_list = []
        #for i in range(0,length-1):
            #print(result_array[length-1])
            #print(item)
        #    norm_list.append(np.around(np.linalg.norm(result_array[i]-result_array[length-1]),decimals = 100))
            #print(temp)
        #length = len(cosine_similarity)
        #print(cosine_similarity.describe)
        #print(cosine_similarity)
        #avg_norm=np.around(np.mean(norm_list), decimals=100) 
        #print(avg_norm)
        #return avg_norm, result_array

        """
        算法2：余弦相似度
        """
        from sklearn.metrics.pairwise import cosine_similarity
        cosine_similarity=np.around(cosine_similarity(result_array),decimals=100)
        cosine_similarity = pd.DataFrame(cosine_similarity)
        avg_cosine=np.mean(cosine_similarity[length-1])
        #print(avg_cosine)
        return avg_cosine

        """
        算法3：几何距离
        """
        #norm_list = []
        #for i in range(0,length-1):
        #    item = result_array[i]
        #    temp=item-result_array[length-1]
        #    norm_list.append(np.around(np.linalg.norm(temp),decimals=100))
        #length = len(cosine_similarity)
        #print(cosine_similarity.describe)
        #print(cosine_similarity)
        #from scipy.stats import gmean
        #avg_norm_g=gmean(norm_list)
        #return avg_norm_g
    return result1


## 是否驳回 person_approve_feature(df,id_person,approve)

In [17]:
# 报销人数据——是否有过被驳回经历
"""
id_person: 员工所在列的列名
approve: 是否同意列的列名

输出：是否同意列部分dataframe
"""
def person_approve_feature(df,id_person,approve):
    df_copy = df.copy()
    def whether_all_approved(df):
        list_neg = ['Submitted & Pending Approval','Approved & In Accounting Review','Not Submitted', 'Sent Back to Employee']
        df_list=list(df)
        number1 = df_list.count("Submitted & Pending Approval")
        number2 = df_list.count('Approved & In Accounting Review')
        number3 = df_list.count('Not Submitted')
        number4 = df_list.count('Sent Back to Employee')
        total = number1+number2+number3+number4
        if total != 0:
            return total
        else:
            return 0

    def whether_all_approved_ratio(df):
        list_neg = ['Submitted & Pending Approval','Approved & In Accounting Review','Not Submitted', 'Sent Back to Employee']
        df_list=list(df)
        number1 = df_list.count("Submitted & Pending Approval")
        number2 = df_list.count('Approved & In Accounting Review')
        number3 = df_list.count('Not Submitted')
        number4 = df_list.count('Sent Back to Employee')
        total = number1+number2+number3+number4
        result = total/len(df_list)
        return result

    # 类型部分
    Employ_Behavior_Approval_Status=df_copy.groupby(id_person)[approve].agg([whether_all_approved,whether_all_approved_ratio])

    Employ_Behavior_Approval_Status=Employ_Behavior_Approval_Status.rename(columns={'whether_all_approved':"Status_Not_Approved_Num",'whether_all_approved_ratio':'Status_Not_Approved_Ratio'})
    return Employ_Behavior_Approval_Status

## 报销参与人数据部分 person_attendee_feature(df,id_person,attendee)

In [9]:
# 行为人数据——报销参与人数据部分
def person_attendee_feature(df,id_person,attendee):
    df_copy = df.copy()
    Employ_Behavior_Attendee_Amount=df_copy.groupby(id_person)[attendee].agg(["median",'std','max'])
    Employ_Behavior_Attendee_Amount=Employ_Behavior_Attendee_Amount.rename(columns={'median': 'Reimbursement_Attendee_Median','std':'Reimbursement_Attendee_Std','max':'Reimbursement_Attendee_max'})
    return Employ_Behavior_Attendee_Amount

## 时间部分特征

### 开始日期、总计报销时长、平均时间间隔 person_time_feature(df,id_person,datetime)

In [10]:
# 行为人数据——时间部分特征
"""
id_person: 员工所在列的列名
datetime: 日期列的列名

输出：日期列部分dataframe
"""
def person_time_feature(df,id_person,datetime):
    df_copy = df.copy()
    def range_date(df):
        result = (df.max()-df.min()).days
        return result


    def avg_range_date(df):
        result = (df.max()-df.min()).days/(df.count()-1)
        return result

    # 类型部分
    df_copy['datetime']=pd.to_datetime(company[df_copy])
    Employ_Behavior_Datetime=df_copy.groupby(id_person)['datetime'].agg(["min",range_date,avg_range_date])

    Employ_Behavior_Datetime=Employ_Behavior_Datetime.rename(columns={'min':"Beginning_Date",'range_date':"Reimbursement_Day_Range",'avg_range_date':"Reimbursement_Avg_Interval"})
    return Employ_Behavior_Datetime

### 每月的平均数据 person_month_statistic(df,id_person,datetime)

In [13]:
# 报销人数据——每月的平均时间数据
"""
id_person: 员工所在列的列名
datetime: 日期列的列名

输出：日期列部分dataframe
"""
def person_month_statistic(df,id_person,datetime):
    company = df.copy()
    # 按月统计
    company_copy=company.copy()
    company_copy['datetime']=pd.to_datetime(company[datetime])
    company_copy.set_index('datetime')
    company_copy=company_copy.set_index('datetime')
    def month_statistic(df):
        result=df.resample('M').sum()
        result_performance = result.mean()
        return result_performance

    def month_statistic_2(df):
        result=df.resample('M').sum()
        result_performance = result.std()
        return result_performance

    def month_statistic_3(df):
        result=df.resample('M').count()
        result_performance = result.mean()
        return result_performance

    def month_statistic_4(df):
        result=df.resample('M').count()
        result_performance = result.std()
        return result_performance

    Employ_Behavior_Datetime_avg=company_copy.groupby(id_person)['Approved_Amount'].agg([month_statistic_3,month_statistic_4,month_statistic,month_statistic_2])

    Employ_Behavior_Datetime_avg=Employ_Behavior_Datetime_avg.rename(columns={'month_statistic_3':"Avg_Count_Per_Month","month_statistic_4":"Std_Count_Per_Month",'month_statistic':"Avg_Per_Month",'month_statistic_2':"Std_Per_Month"})
    return Employ_Behavior_Datetime_avg


## 报销频率，报销速度, 分部门，与报销类型 person_add_other_feature(df)

In [11]:
# 报销人数据——其他特征数据
def person_add_other_feature(df):
    Employ_Behavior = df.copy()
    # 报销频率
    Employ_Behavior['Reimbursement_Frequency']=Employ_Behavior['Total_Reimbursement_Times']/Employ_Behavior['Reimbursement_Day_Range']
    # 报销平均时长
    Employ_Behavior['Reimbursement_Avg_Amount_Daily']=Employ_Behavior['Total_Amount']/Employ_Behavior['Reimbursement_Day_Range']
    return Employ_Behavior

In [18]:
list_cols=['客户娱乐', '加班餐补', '会议', '租车费', '住宿', 'HCP支付', '机票费',
       '翻译费', '地面交通', '停车费', '员工餐费', '运输费', '金融其他', '员工娱乐', '其他']


In [67]:
Employ_Behavior_Expense_type = Employ_Behavior[['客户娱乐', '加班餐补', '会议', '租车费', '住宿', 'HCP支付', '机票费',
       '翻译费', '地面交通', '停车费', '员工餐费', '运输费', '金融其他', '员工娱乐', '其他','BU']]

In [72]:
test_others=Employ_Behavior_Expense_type[Employ_Behavior_Expense_type['BU']=='其他部门']
test_sales=Employ_Behavior_Expense_type[Employ_Behavior_Expense_type['BU']=='销售']
test_purchase=Employ_Behavior_Expense_type[Employ_Behavior_Expense_type['BU']=='采购']
test_market=Employ_Behavior_Expense_type[Employ_Behavior_Expense_type['BU']=='市场']

In [81]:
test_others_b=Employ_Behavior[Employ_Behavior['BU']=='其他部门']
test_sales_b=Employ_Behavior[Employ_Behavior['BU']=='销售']
test_purchase_b=Employ_Behavior[Employ_Behavior['BU']=='采购']
test_market_b=Employ_Behavior[Employ_Behavior['BU']=='市场']

# 异常值检测

## 孤立森林 

In [2]:
def IsolationForest(df,df2,list_col,contamination):
    
    from sklearn.ensemble import IsolationForest

    rng = np.random.RandomState(42)

    df_n = df[list_col]

    cov = IsolationForest(n_estimators=100,behaviour='new', max_samples=256,random_state=rng, contamination=contamination).fit(df_n)

    test_result = cov.predict(df_n)

    print('预测完成')
    x_test_copy = df2.copy()
    x_test_copy['label'] = test_result
    normal_test_data = x_test_copy[x_test_copy['label'] == 1]
    abnormal_test_data = x_test_copy[x_test_copy['label'] == -1]
#     abnormal_test_data.to_csv(r"abnnormal_test_data.csv",encoding='gbk')
#     abnormal_test_data.describe().to_csv(r"abnnormal_test_data_des",encoding='gbk')
    return abnormal_test_data


# 参数解释
class sklearn.ensemble.IsolationForest（n_estimators = 100，max_samples ='auto'，contamination ='legacy'，max_features = 1.0，bootstrap = False，n_jobs = None，behavior ='old'，random_state = None，verbose = 0 ）

n_estimators ： int，optional（默认值= 100）
整体中基本估算器的数量。配置iTree树的多少
 
max_samples ： int或float，optional（default =“auto”）
从X中抽取的样本数量，用于训练每个基本估算器。
如果是int，则绘制max_samples样本。
如果是float，则绘制max_samples * X.shape [0]样本。
如果是“auto”，则max_samples = min（256，n_samples）。
如果max_samples大于提供的样本数，则所有样本将用于所有树（无采样）。
 
contamination ： float（0.，0.5），可选（默认值= 0.1）
数据集的污染量，即数据集中异常值的比例。在拟合时用于定义决策函数的阈值。如果是“自动”，则确定决策函数阈值，如原始论文中所示。
 
在版本0.20中更改：默认值contamination将从0.20更改为'auto'0.22。
 
max_features ： int或float，可选（默认值= 1.0）
从X绘制以训练每个基本估计器的特征数。
 
如果是int，则绘制max_features特征。
如果是float，则绘制max_features * X.shape [1]特征。
bootstrap ： boolean，optional（default = False）
如果为True，则单个树适合于通过替换采样的训练数据的随机子集。如果为假，则执行未更换的采样。
 
n_jobs ： int或None，可选（默认=无）
适合和预测并行运行的作业数。 None除非在joblib.parallel_backend上下文中，否则表示1 。 -1表示使用所有处理器。
 
random_state ： int，RandomState实例或None，可选（默认=无）
如果是int，则random_state是随机数生成器使用的种子; 如果是RandomState实例，则random_state是随机数生成器; 如果没有，随机数生成器所使用的RandomState实例np.random。
 
verbose ： int，optional（默认值= 0）
控制树构建过程的详细程度。

## LoF

In [1]:
#LOF
def localoutlierfactor(data, predict, k):
    from sklearn.neighbors import LocalOutlierFactor
    clf = LocalOutlierFactor(n_neighbors=k + 1, algorithm='auto', contamination=0.1, n_jobs=-1)
    clf.fit(data)
    #     # 记录 k 邻域距离
    predict['k distances'] = clf.kneighbors(predict)[0].max(axis=1)
    # 记录 LOF 离群因子，做相反数处理
    predict['local outlier factor'] = -clf._decision_function(predict.iloc[:, :-1])
    return predict


def plot_lof(result, method):
    import matplotlib.pyplot as plt
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
    plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
    plt.figure(figsize=(8, 4)).add_subplot(111)
    plt.scatter(result[result['local outlier factor'] > method].index,
                result[result['local outlier factor'] > method]['local outlier factor'], c='red', s=50,
                marker='.', alpha=None,
                label='离群点')
    plt.scatter(result[result['local outlier factor'] <= method].index,
                result[result['local outlier factor'] <= method]['local outlier factor'], c='black', s=50,
                marker='.', alpha=None, label='正常点')
    plt.hlines(method, -2, 2 + max(result.index), linestyles='--')
    plt.xlim(-2, 2 + max(result.index))
    plt.title('LOF局部离群点检测', fontsize=13)
    plt.ylabel('局部离群因子', fontsize=15)
    plt.legend()
    plt.show()


def lof(data1, data2,list_col, predict=None, k=5):
    data = data1[list_col]
    import pandas as pd
    # 判断是否传入测试数据，若没有传入则测试数据赋值为训练数据
    try:
        if predict == None:
            predict = data.copy()
    except Exception:
        pass
    predict = pd.DataFrame(predict)
    # 计算 LOF 离群因子
    predict = localoutlierfactor(data, predict, k)
    print('计算离群因子完成')
    print('  ')
    data2['local outlier factor'] =  predict['local outlier factor']

    # 根据阈值划分离群点与正常点
    outliers = data2[data2['local outlier factor'] > 1].sort_values(by='local outlier factor')
    inliers = data2[data2['local outlier factor'] <= 1].sort_values(by='local outlier factor')
    outliers.to_csv(r"outliers.csv",encoding='gbk')
    outliers.describe().to_csv(r"outlier_des.csv",encoding='gbk')
    return outliers,inliers
